# Fusion des bases de données

Dans ce notebook, on fusionne toutes nos bases de données pour créer notre base de données de travail.

In [1]:
import numpy as np
import pandas as pd

In [3]:
#On importe le notebook dans lequel on a récupéré et nettoyé nos données
%run Recuperation_donnees.ipynb

## Fusion des données et création de la base

On commence par ajouter une variable avec la population, une variable avec le PIB par habitant, une variable avec l'investissement dans le sport et une variable avec l'espérance de vie à la naissance pour l'année correspondante, à la base des médailles.

In [14]:
#Fonction qui, pour une année donnée, fait une jointure entre la base des médailles et la population, le PIB par habitant,
#l'investissement dans le sport et l'espérance de vie, pour cette année donnée

def fonction_jointure(annee):
    
    df_medailles_jointure = df_medailles_copy[df_medailles_copy['Annee'] == annee]
    
    variable_annee_pop = str(annee) + " [YR" + str(annee) + "]"
    df_pop_jointure = df_pop[['Country Code', variable_annee_pop]]
    
    variable_annee_pib = str(annee) + " [YR" + str(annee) + "]"
    df_pib_jointure = df_pib_hab[['Country Code', variable_annee_pib]]
    
    #Pour l'espérance de vie, si l'année c'est 2020, on prend l'espérance de vie en 2019 car il n'y a pas de données pour 2020
    variable_annee_esperance_vie = str(annee) + " [YR" + str(annee) + "]"
    df_esperance_vie_jointure = df_esperance_vie[['Country Code', variable_annee_esperance_vie]]
    df_esperance_vie_jointure2020 = df_esperance_vie[['Country Code', "2019 [YR2019]"]]
    
    variable_annee_sport = str(annee)
    df_invest_sport_jointure = df_invest_sport_merged[['Country Code', variable_annee_sport]]
    
    df_merged1 = df_medailles_jointure.merge(df_pop_jointure, on="Country Code")
    df_merged1.rename({variable_annee_pop: "Population"}, axis=1, inplace=True)
    
    df_merged2 = df_merged1.merge(df_pib_jointure, on="Country Code")
    df_merged2.rename({variable_annee_pib: "PIB par hab"}, axis=1, inplace=True)
    
    if annee == 2020:
        df_merged3 = df_merged2.merge(df_esperance_vie_jointure2020, on="Country Code")
        df_merged3.rename({"2019 [YR2019]": "Esperance de vie à naissance"}, axis=1, inplace=True)
    else:
        df_merged3 = df_merged2.merge(df_esperance_vie_jointure, on="Country Code")
        df_merged3.rename({variable_annee_esperance_vie: "Esperance de vie à naissance"}, axis=1, inplace=True)
    
    #Pour l'investissement dans la sport, comme il y a peu de données, on fait une jointure à gauche
    df_merged = df_merged3.merge(df_invest_sport_jointure, how="left", on="Country Code")
    df_merged.rename({variable_annee_sport: "Investissement sport"}, axis=1, inplace=True)
    
    return(df_merged)   

In [17]:
#on teste la fonction
fonction_jointure(2020).head(10)

,Country Name,Country Code,Gold,Silver,Bronze,Total,Annee,Lieu,Pays_hote,Code_pays_hote,ISO 3166-1,Population,PIB par hab,Esperance de vie à naissance,Investissement sport
0,United States,USA,39,41,33,113,2020,Tokyo,Japan,JPN,USA,3.294841e+08,63413.513858,78.787805,NaN
1,People's Republic of China,CHN,38,32,18,88,2020,Tokyo,Japan,JPN,CHN,1.410929e+09,10434.775187,76.912000,NaN
2,Japan,JPN,27,14,17,58,2020,Tokyo,Japan,JPN,JPN,1.258360e+08,40193.252445,84.356341,NaN
3,Great Britain,GBR,22,21,22,65,2020,Tokyo,Japan,JPN,GBR,6.721529e+07,41124.534769,81.204878,NaN
4,Australia,AUS,17,7,22,46,2020,Tokyo,Japan,JPN,AUS,2.568704e+07,51692.842748,82.900000,17332.00
5,France,FRA,10,12,11,33,2020,Tokyo,Japan,JPN,FRA,6.739158e+07,39030.360371,82.578049,NaN
6,Italy,ITA,10,10,20,40,2020,Tokyo,Japan,JPN,ITA,5.955402e+07,31714.220947,83.197561,NaN
7,Canada,CAN,7,6,11,24,2020,Tokyo,Japan,JPN,CAN,3.800524e+07,43258.176320,82.048780,20729.00
8,Brazil,BRA,7,6,8,21,2020,Tokyo,Japan,JPN,BRA,2.125594e+08,6796.844542,75.881000,NaN
9,New Zealand,NZL,7,6,7,20,2020,Tokyo,Japan,JPN,NZL,5.084300e+06,41441.466667,81.707317,3090.66


In [18]:
#On construit la base dans laquelle on a la population, le PIB par habitant, l'investissemet dans le sport et l'espérance de vie
# à la naissance, pour l'année en cours

df = pd.DataFrame(columns=[])

for annee in df_medailles_copy['Annee'].unique():
    df_merged = fonction_jointure(annee)
    df = pd.concat([df, df_merged])

df

,Country Name,Country Code,Gold,Silver,Bronze,Total,Annee,Lieu,Pays_hote,Code_pays_hote,ISO 3166-1,Population,PIB par hab,Esperance de vie à naissance,Investissement sport
0,United States,USA,37,34,37,108,1992,Barcelona,Spain,ESP,USA,2.565140e+08,25418.990776,75.617073,19250.647
1,People's Republic of China,CHN,16,22,16,54,1992,Barcelona,Spain,ESP,CHN,1.164970e+09,366.460692,69.355000,NaN
2,Cuba,CUB,14,6,11,31,1992,Barcelona,Spain,ESP,CUB,1.073639e+07,2057.103595,74.801000,NaN
3,Spain,ESP,13,7,2,22,1992,Barcelona,Spain,ESP,ESP,3.915768e+07,16112.188915,77.410000,NaN
4,Republic of Korea,KOR,12,5,12,29,1992,Barcelona,Spain,ESP,KOR,4.374796e+07,8126.670390,72.497561,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Finland,FIN,0,0,2,2,2020,Tokyo,Japan,JPN,FIN,5.530719e+06,48773.281169,81.785366,NaN
60,Côte d'Ivoire,CIV,0,0,1,1,2020,Tokyo,Japan,JPN,CIV,2.637828e+07,2325.723705,57.783000,NaN
61,Ghana,GHA,0,0,1,1,2020,Tokyo,Japan,JPN,GHA,3.107294e+07,2205.529016,64.074000,NaN
62,Republic of Moldova,MDA,0,0,1,1,2020,Tokyo,Japan,JPN,MDA,2.620495e+06,4547.059721,71.901000,1733.200


On ajoute une variable indicatrice indiquant si le pays est le pays hôte ou pas.

In [19]:
df['Pays_hote_oui_non'] = (df['Country Code'] == df['Code_pays_hote'])
df['Pays_hote_oui_non'] = df['Pays_hote_oui_non'].astype(int)
df

,Country Name,Country Code,Gold,Silver,Bronze,Total,Annee,Lieu,Pays_hote,Code_pays_hote,ISO 3166-1,Population,PIB par hab,Esperance de vie à naissance,Investissement sport,Pays_hote_oui_non
0,United States,USA,37,34,37,108,1992,Barcelona,Spain,ESP,USA,2.565140e+08,25418.990776,75.617073,19250.647,0
1,People's Republic of China,CHN,16,22,16,54,1992,Barcelona,Spain,ESP,CHN,1.164970e+09,366.460692,69.355000,NaN,0
2,Cuba,CUB,14,6,11,31,1992,Barcelona,Spain,ESP,CUB,1.073639e+07,2057.103595,74.801000,NaN,0
3,Spain,ESP,13,7,2,22,1992,Barcelona,Spain,ESP,ESP,3.915768e+07,16112.188915,77.410000,NaN,1
4,Republic of Korea,KOR,12,5,12,29,1992,Barcelona,Spain,ESP,KOR,4.374796e+07,8126.670390,72.497561,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Finland,FIN,0,0,2,2,2020,Tokyo,Japan,JPN,FIN,5.530719e+06,48773.281169,81.785366,NaN,0
60,Côte d'Ivoire,CIV,0,0,1,1,2020,Tokyo,Japan,JPN,CIV,2.637828e+07,2325.723705,57.783000,NaN,0
61,Ghana,GHA,0,0,1,1,2020,Tokyo,Japan,JPN,GHA,3.107294e+07,2205.529016,64.074000,NaN,0
62,Republic of Moldova,MDA,0,0,1,1,2020,Tokyo,Japan,JPN,MDA,2.620495e+06,4547.059721,71.901000,1733.200,0


On crée des variables indicatrices pour l'année.

In [20]:
df_dummies = pd.get_dummies(df, columns=["Annee"])
df_dummies

,Country Name,Country Code,Gold,Silver,Bronze,Total,Lieu,Pays_hote,Code_pays_hote,ISO 3166-1,...,Investissement sport,Pays_hote_oui_non,Annee_1992,Annee_1996,Annee_2000,Annee_2004,Annee_2008,Annee_2012,Annee_2016,Annee_2020
0,United States,USA,37,34,37,108,Barcelona,Spain,ESP,USA,...,19250.647,0,1,0,0,0,0,0,0,0
1,People's Republic of China,CHN,16,22,16,54,Barcelona,Spain,ESP,CHN,...,NaN,0,1,0,0,0,0,0,0,0
2,Cuba,CUB,14,6,11,31,Barcelona,Spain,ESP,CUB,...,NaN,0,1,0,0,0,0,0,0,0
3,Spain,ESP,13,7,2,22,Barcelona,Spain,ESP,ESP,...,NaN,1,1,0,0,0,0,0,0,0
4,Republic of Korea,KOR,12,5,12,29,Barcelona,Spain,ESP,KOR,...,NaN,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Finland,FIN,0,0,2,2,Tokyo,Japan,JPN,FIN,...,NaN,0,0,0,0,0,0,0,0,1
60,Côte d'Ivoire,CIV,0,0,1,1,Tokyo,Japan,JPN,CIV,...,NaN,0,0,0,0,0,0,0,0,1
61,Ghana,GHA,0,0,1,1,Tokyo,Japan,JPN,GHA,...,NaN,0,0,0,0,0,0,0,0,1
62,Republic of Moldova,MDA,0,0,1,1,Tokyo,Japan,JPN,MDA,...,1733.200,0,0,0,0,0,0,0,0,1
